In [ ]:
import torch
import torch.nn.functional as F
import plotly.express as px
import pandas as pd

# Linear Reagressiom

## Setup

In [ ]:
torch.random.manual_seed(42)

# generate data with noise
x = torch.linspace(-10, 10, 200)


def f(x: torch.Tensor) -> torch.Tensor:
    return -2 * x + 3


y_noise = f(x) + torch.randn_like(x) * 2


def f_param(x: torch.Tensor, m: torch.Tensor, c: torch.Tensor) -> torch.Tensor:
    return m * x + c


# randomly initialize the slope and intercept
slope = torch.scalar_tensor((torch.randn(1).item() * -2), requires_grad=True)
intercept = torch.scalar_tensor((torch.randn(1).item() * 2), requires_grad=True)


# define the loss function
def lin_loss(
    x: torch.Tensor, y: torch.Tensor, m: torch.Tensor, c: torch.Tensor
) -> torch.Tensor:
    return F.mse_loss(f_param(x, m, c), y)


# define the optimizer
optimizer = torch.optim.Adam((slope, intercept), lr=0.01)

print(slope, intercept)

## Training Loop

In [ ]:
# hyperparameters
epochs = 1000

# plot parameters
losses = []
ys = []

# training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = lin_loss(x, y_noise, slope, intercept)
    loss.backward()
    if epoch % 10 == 0:
        losses.append(loss.item())
        ys.append(f_param(x, slope, intercept))
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

## Plots

In [ ]:
data = []

for i in range(len(ys)):
    for j in range(len(x)):
        data.append(
            {"x": x[j].detach().numpy(), "y": ys[i][j].detach().numpy(), "frame": i + 1}
        )
df = pd.DataFrame(data)

fig = px.line(df, x="x", y="y", animation_frame="frame", title="Linear Regression")
fig.add_trace(px.scatter(x=x.detach().numpy(), y=y_noise.detach().numpy()).data[0])
fig.update_traces(marker=dict(color="red"))
fig.show()

In [ ]:
fig = px.line(x=range(len(losses)), y=losses, title="Loss every 10 epochs")
fig.update_xaxes(title="Epochs * 10")
fig.update_yaxes(title="Loss")

# Quadratic Regression

## Setup

In [ ]:
torch.random.manual_seed(42)

# generate data with noise
x = torch.linspace(-10, 10, 200)


def f(x: torch.Tensor) -> torch.Tensor:
    return -2.0 * x**2 + 3 * x + 1


y_noise = f(x) + torch.randn_like(x) * 10


def f_param(
    x: torch.Tensor, a: torch.Tensor, b: torch.Tensor, c: torch.Tensor
) -> torch.Tensor:
    return a * x**2 + b * x + c


a = torch.scalar_tensor((torch.randn(1).item()) * -1, requires_grad=True)
b = torch.scalar_tensor((torch.randn(1).item()), requires_grad=True)
c = torch.scalar_tensor((torch.randn(1).item()), requires_grad=True)


def quad_loss(
    x: torch.Tensor, y: torch.Tensor, a: torch.Tensor, b: torch.Tensor, c: torch.Tensor
) -> torch.Tensor:
    return F.mse_loss(f_param(x, a, b, c), y)


optimizer = torch.optim.Adam((a, b, c), lr=0.01)

print(a, b, c)

## Training Loop

In [ ]:
# hyperparameters
epochs = 1000

# plot parameters
losses = []
ys = []

# training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = quad_loss(x, y_noise, a, b, c)
    loss.backward()
    if epoch % 10 == 0:
        losses.append(loss.item())
        ys.append(f_param(x, a, b, c))
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

## Plots

In [ ]:
data = []

for i in range(len(ys)):
    for j in range(len(x)):
        data.append(
            {"x": x[j].detach().numpy(), "y": ys[i][j].detach().numpy(), "frame": i + 1}
        )

df = pd.DataFrame(data)

fig = px.line(df, x="x", y="y", animation_frame="frame", title="Quadratic Regression")
fig.add_trace(px.scatter(x=x.detach().numpy(), y=y_noise.detach().numpy()).data[0])
fig.update_traces(marker=dict(color="red"))
fig.show()

In [ ]:
fig = px.line(x=range(len(losses)), y=losses, title="Loss every 10 epochs")
fig.update_xaxes(title="Epochs * 10")
fig.update_yaxes(title="Loss")